P317 Example 8.1

In [1]:
import numpy as np

In [2]:
N = 3
K = 3
mu = np.array([0.8, 0.6, 0.4])
m = np.array([2, 3, 1])
alpha = np.array([1.0, 0.667, 0.2])
rho = alpha / mu

In [3]:
def fact(n):
    if n <= 1:
        return 1
    return n * fact(n-1)

In [4]:
def getBeta(n, k): #P289 式 7.62
    if m[n] == 1:
        return 1
    elif k <= m[n]:
        return fact(k)
    elif k >= m[n]:
        return fact(m[n]) * m[n]**(k-m[n])

In [5]:
def getF(n, k):#n:ノード番号は1引いた数字
    f = (alpha[n]/mu[n])**k
    f /= getBeta(n, k)
    return f

In [6]:
for n in range(N):
    for k in range(K+1): 
        print('n = {0}, k = {1}, F = {2}'.format(n, k, getF(n, k)))

n = 0, k = 0, F = 1.0
n = 0, k = 1, F = 1.25
n = 0, k = 2, F = 0.78125
n = 0, k = 3, F = 0.48828125
n = 1, k = 0, F = 1.0
n = 1, k = 1, F = 1.1116666666666668
n = 1, k = 2, F = 0.617901388888889
n = 1, k = 3, F = 0.22896679243827167
n = 2, k = 0, F = 1.0
n = 2, k = 1, F = 0.5
n = 2, k = 2, F = 0.25
n = 2, k = 3, F = 0.125


In [7]:
def calcConvolution(n, k, rho): #メモ化利用なし
    #rho = alpha / mu
    if k == 0: #Gj(0) = 1 (j = 1,...,N)
        g = 1
    elif n == 0:#G(0,k) = f0(k)
        g = np.power(rho[n], k)
    else : 
        g = calcConvolution(n-1, k,rho) + rho[n] * calcConvolution(n, k-1,rho)
    return g

In [8]:
g = calcConvolution(N-1, K, rho) #ノード番号は1小さくしている
print(g)
#窓口数を考慮できないので値が合わない

9.418018115740743


In [9]:
def getConvolution(n, k, rho): #QN&MC P314 式(8.4),(8.5),(8.6)利用
    if k == 0:
        g = 1
    elif n == 0:
        g = getF(n, k)
    else :
        g = 0
        for i in range(k + 1):
            g += getF(n, i) * getConvolution(n-1, k-i, rho)
    #print('n = {0}, k = {1}, G(n,k) = {2}'.format(n, k, g)) #QN&MC P314 Table8.1
    return g

In [10]:
g = getConvolution(N-1, K, rho) #ノード番号は1小さくしている
print(g)

4.467898389660494


In [11]:
def getMarginalProbabilities(n, k): #QN&MC P316 式(8.13)
    pi = rho[n]**k / getConvolution(N-1, K, rho) * (getConvolution(N-1, K-k, rho) - rho[n] * getConvolution(N-1, K-k-1, rho))
    return pi

In [12]:
for k in range(K+1): #n = Nのとき
    marginal = getMarginalProbabilities(N-1, k) #ノード番号は1小さくしている
    print('pi({0},{1}) = {2}'.format(n, k, marginal))

pi(2,0) = 0.5277905082487573
pi(2,1) = 0.31208573685066865
pi(2,2) = 0.13214639527904107
pi(2,3) = 0.02797735962153304


In [13]:
def getGi(n, k, rho): #QN&MC P316 式(8.11), (8.12)
    if k == 0: #式(8.12)
        gi = 1
    else:
        gi = getConvolution(N-1, k, rho)
        for i in range(1, k + 1):
            gi -= getF(n, i) * getGi(n, k-i, rho)
    return gi

In [14]:
for n in range(N-1): #QN&MC P319確認
    for k in range(K+1):
        gi = getGi(n, k, rho)
        print('G({0},{1}) = {2}'.format(n, k, gi))

G(0,0) = 1
G(0,1) = 1.6116666666666668
G(0,2) = 1.423734722222222
G(0,3) = 0.9408341535493832
G(1,0) = 1
G(1,1) = 1.75
G(1,2) = 1.6562499999999996
G(1,3) = 1.3164062500000002


In [15]:
def getMarginal(): #QN&MC P315 式(8.7)
    mpi = np.zeros((N, K+1)) #周辺分布
    l = np.zeros(N) #平均系内人数
    for n in range(N):#周辺分布計算
        for k in range(K+1):
            mpi[n,k] = getF(n, k) * getGi(n, K - k, rho) / g
    for n in range(N):
        for k in range(K+1):
            l[n] += k * mpi[n,k]
    return mpi, l

In [16]:
mpi, l = getMarginal()
print(mpi)
print(l)

[[0.21057644 0.39832338 0.28181361 0.10928656]
 [0.29463657 0.41209485 0.24202149 0.05124709]
 [0.52779051 0.31208574 0.1321464  0.02797736]]
[1.28981029 1.0498791  0.66031061]


P323 Example 8.2

In [17]:
N = 4
R = 2
K1 = 1
K2 = 2
K = [K1, K2]
mu = np.array([[1/1, 1/2],[1/4, 1/5],[1/8, 1/10],[1/12, 1/16]])
alpha = np.array([[1, 1],[0.4, 0.4],[0.4, 0.3],[0.2, 0.3]])
#Node1:Type2プロセッサシェアリング（Processor Sharing: PS）, Node2:Type4後着順割込継続型（LCFS-PR), Node3:Type4後着順割込継続型（LCFS-PR), Node4:Type3無限サーバ（Infinite Server: IS）, その他Type1(FCFS) 

In [18]:
def getCombiList(combi, K, R, idx): #クラスごとの人数分布算出 K = [K1, K2], idxはどのクラスかを指定
    if len(combi) == R:
        #print(combi)
        exec('combi_list.append({})'.format(combi))
        return 
    for v in range(K[idx]+1):
        combi.append(v)
        getCombiList(combi, K, R, idx+1)
        combi.pop() 

In [19]:
def getCombiList2(combi, K, R, idx, Klist):
    if len(combi) == R:
        #print(combi)
        Klist.append(combi.copy())
        #print(Klist)
        return Klist
    for v in range(K[idx]+1):
        combi.append(v)
        Klist = getCombiList2(combi, K, R, idx+1, Klist)
        combi.pop()
    return Klist

In [20]:
combi_list = []
getCombiList([], K, R, 0)
combi_list

[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2]]

In [21]:
# P323 Table 8.2 Fi(Si)の計算, P303式(7.82)
#Siはノードiにおけるクラス別人数分布:(ノードiのクラス0の人数, ノードiのクラス1の人数)
#Siは(0,0),(1,0),(0,1),(1,1),(0,2),(1,2) K1=1,K2=2なので
def FS(n, alpha, mu, Si, type_number):#sは状態分布, type_number =1(FCFS),2(PS),3(IS),4(LCFS-PR)
    f = 1
    if type_number == 1:
        print('FCFS') #ここはまだ未実装
    else:
        for r in range(R):#type-3はこのループで終わり
            f *= 1 / fact(Si[r]) * (alpha[n,r] / mu[n,r])**Si[r]
        if type_number == 2 or type_number == 4:#type-2,4は累乗をかける    
            f *= fact(sum(Si))
    return f

In [22]:
#ループでの人数分布作成は使わない
#type_number = [2, 4, 4, 3]
#type_list = [2, 4, 4, 3]
#for n in range(N):
#    for k1 in range(K1+1): #kのリストを作成
#        for k2 in range(K2+1):
#            f = FS(n, alpha, mu, [k1, k2], type_number[n])
#            print('F{0}([{1},{2}]) = {3}'.format(n, k1, k2, f))

In [23]:
#ループではなく、getCombiListを利用
type_number = [2, 4, 4, 3]
type_list = [2, 4, 4, 3]
combi_list = []
combi_list = getCombiList2([], K, R, 0, combi_list)
for n in range(N):
    for k in combi_list:
        f = FS(n, alpha, mu, k, type_number[n])
        print('F{0}([{1}]) = {2}'.format(n, k, f))

F0([[0, 0]]) = 1.0
F0([[0, 1]]) = 2.0
F0([[0, 2]]) = 4.0
F0([[1, 0]]) = 1.0
F0([[1, 1]]) = 4.0
F0([[1, 2]]) = 12.0
F1([[0, 0]]) = 1.0
F1([[0, 1]]) = 2.0
F1([[0, 2]]) = 4.0
F1([[1, 0]]) = 1.6
F1([[1, 1]]) = 6.4
F1([[1, 2]]) = 19.200000000000003
F2([[0, 0]]) = 1.0
F2([[0, 1]]) = 2.9999999999999996
F2([[0, 2]]) = 8.999999999999998
F2([[1, 0]]) = 3.2
F2([[1, 1]]) = 19.2
F2([[1, 2]]) = 86.39999999999999
F3([[0, 0]]) = 1.0
F3([[0, 1]]) = 4.8
F3([[0, 2]]) = 11.52
F3([[1, 0]]) = 2.4000000000000004
F3([[1, 1]]) = 11.520000000000001
F3([[1, 2]]) = 27.648000000000003


In [24]:
#ループでの人数分布作成は使わない
#def getConvolution(n, alpha, mu, Si, type_num):#P321式(8.21)
#    g = 0
#    type_list = [2, 4, 4, 3] #type_listに名前変更(type_numberはFSの時は番号が変わらないのでいいが、再帰の時は番号が変わるのでtype_listと変えた。FSもtype_listとして処理した方が混乱しなくていいかも)
#    if n == 0:
#        g = FS(n, alpha, mu, Si, type_num)#
#    elif n >= 1:
#        for i in range(Si[0]+1):#課題：今回は2クラスなので２重ループでよかった。多クラスの対応を考える
#            for j in range(Si[1]+1):#解決法：重複組合せをリストで返す関数を呼び出し、そのリストをループで回せば良い
#                g += getConvolution(n-1, alpha, mu, [i,j], type_list[n-1]) * FS(n, alpha, mu, [Si[0]-i, Si[1]-j], type_num) #ここも多クラスの時の課題
#    return g    

In [25]:
def getConvolution(n, alpha, mu, Si, type_num):#P321式(8.21)
    g = 0
    type_list = [2, 4, 4, 3] #type_listに名前変更(type_numberはFSの時は番号が変わらないのでいいが、再帰の時は番号が変わるのでtype_listと変えた。FSもtype_listとして処理した方が混乱しなくていいかも)
    combi_list = []
    combi_list = getCombiList2([], Si, R, 0, combi_list)
    if n == 0:
        g = FS(n, alpha, mu, Si, type_num)#
    elif n >= 1:
        for k in combi_list:
            g += getConvolution(n-1, alpha, mu, k, type_list[n-1]) * FS(n, alpha, mu, np.array(Si)-np.array(k), type_num) #ここも多クラスの時の課題
    return g    

In [26]:
#GK = 0
#for n in range(N): #P324 Table 8.3
#    for k1 in range(K1+1):
#        for k2 in range(K2+1):
#            g = getConvolution(n, alpha, mu, [k1,k2], type_list[n])
#            print('G{0}({1},{2}) = {3}'.format(n,k1,k2,g))
#            if n == N-1 and k1 == K1 and k2 == K2: #定常分布で使うためGKを確保
#                GK = g
#print(GK)

In [27]:
GK = 0
combi_list = []
combi_list = getCombiList2([], K, R, 0, combi_list)
for n in range(N): #P324 Table 8.3
    for i, k in enumerate(combi_list):
        g = getConvolution(n, alpha, mu, k, type_list[n])
        print('G{0}({1}) = {2}'.format(n,k,g))
        #if n == N-1 and k[0] == K1 and k[1] == K2: #定常分布で使うためGKを確保
        if n == N-1 and i == len(combi_list)-1: #combi_listの最後の要素のとき
            GK = g
print(GK)

G0([0, 0]) = 1.0
G0([0, 1]) = 2.0
G0([0, 2]) = 4.0
G0([1, 0]) = 1.0
G0([1, 1]) = 4.0
G0([1, 2]) = 12.0
G1([0, 0]) = 1.0
G1([0, 1]) = 4.0
G1([0, 2]) = 12.0
G1([1, 0]) = 2.6
G1([1, 1]) = 15.600000000000001
G1([1, 2]) = 62.4
G2([0, 0]) = 1.0
G2([0, 1]) = 7.0
G2([0, 2]) = 33.0
G2([1, 0]) = 5.800000000000001
G2([1, 1]) = 55.4
G2([1, 2]) = 334.2
G3([0, 0]) = 1.0
G3([0, 1]) = 11.8
G3([0, 2]) = 78.12
G3([1, 0]) = 8.200000000000001
G3([1, 1]) = 111.56
G3([1, 2]) = 854.424
854.424


In [28]:
#def getGi(n, Si, alpha, mu): #P322 式(8.25,26,27) 引数nはG_N^(i)のiに対応する
#    if sum(Si) == 0: #式(8.27)
#        gi = 1
#    else:
#        gi = getConvolution(N-1, alpha, mu, Si, type_list[N-1])
#        for k1 in range(Si[0]+1): #Siになるまでの全ての組み合わせ(課題：多クラスにも対応できるようにする)
#            for k2 in range(Si[1]+1):
#                if(k1 + k2 == 0):#j=0の場合は除く(8.26) 
#                    continue
#                else:
#                    gi -= FS(n, alpha, mu, [k1, k2], type_list[n]) * getGi(n, [Si[0]-k1, Si[1]-k2], alpha, mu)
#    return gi

In [29]:
def getGi(n, Si, alpha, mu): #P322 式(8.25,26,27) 引数nはG_N^(i)のiに対応する
    if sum(Si) == 0: #式(8.27)
        gi = 1
    else:
        gi = getConvolution(N-1, alpha, mu, Si, type_list[N-1])
        combi_list = []
        combi_list = getCombiList2([], Si, R, 0, combi_list)
        for k in combi_list:
            if(sum(k) == 0):#j=0の場合は除く(8.26) 
                continue
            else:
                gi -= FS(n, alpha, mu, k, type_list[n]) * getGi(n, np.array(Si)-np.array(k), alpha, mu)
    return gi

In [30]:
#for n in range(N):
#    for k1 in range(K1+1):
#        for k2 in range(K2+1):
#            gi = getGi(n, [k1, k2], alpha, mu)
#            print('GN({0})({1},{2}) = {3}'.format(n, k1, k2, gi))
#最終ノードを除いた場合GN(3)(0,0) = 1 ~は数値OKだが、他のノードを除いた場合は確認できない

In [31]:
combi_list = []
combi_list = getCombiList2([], K, R, 0, combi_list)
for n in range(N):
    for k in combi_list:
        gi = getGi(n, k, alpha, mu)
        print('GN({0})({1}) = {2}'.format(n, k, gi))
#最終ノードを除いた場合GN(3)(0,0) = 1 ~は数値OKだが、他のノードを除いた場合は確認できない

GN(0)([0, 0]) = 1
GN(0)([0, 1]) = 9.8
GN(0)([0, 2]) = 54.52
GN(0)([1, 0]) = 7.200000000000001
GN(0)([1, 1]) = 83.36
GN(0)([1, 2]) = 553.184
GN(1)([0, 0]) = 1
GN(1)([0, 1]) = 9.8
GN(1)([0, 2]) = 54.52
GN(1)([1, 0]) = 6.600000000000001
GN(1)([1, 1]) = 76.27999999999999
GN(1)([1, 2]) = 506.31200000000007
GN(2)([0, 0]) = 1
GN(2)([0, 1]) = 8.8
GN(2)([0, 2]) = 42.720000000000006
GN(2)([1, 0]) = 5.000000000000001
GN(2)([1, 1]) = 49.2
GN(2)([1, 2]) = 269.7599999999999
GN(3)([0, 0]) = 1
GN(3)([0, 1]) = 7.000000000000001
GN(3)([0, 2]) = 33.0
GN(3)([1, 0]) = 5.800000000000001
GN(3)([1, 1]) = 55.399999999999984
GN(3)([1, 2]) = 334.20000000000005


In [32]:
#定常分布を求める
#G(K)をgetConvolutionでN=n-1、Si=[K1,K2]となった時がGKなので最初にそれを取得してから(8.23)を計算する
#def getMarginalProbability(n, Si, alpha, mu):
#    mp = FS(n, alpha, mu, Si, type_list[n]) * getGi(n, [K1-Si[0], K2-Si[1]], alpha, mu) / GK
#    return mp

In [33]:
#定常分布を求める
#G(K)をgetConvolutionでN=n-1、Si=[K1,K2]となった時がGKなので最初にそれを取得してから(8.23)を計算する
def getMarginalProbability(n, Si, alpha, mu):
    mp = FS(n, alpha, mu, Si, type_list[n]) * getGi(n, np.array(K)-np.array(Si), alpha, mu) / GK
    return mp

In [34]:
#for n in range(N):
#    for k1 in range(K1+1):
#        for k2 in range(K2+1):
#            mp = getMarginalProbability(n, [k1, k2], alpha, mu)
#            print('mp({0})({1},{2}) = {3}'.format(n, k1, k2, mp))

In [35]:
combi_list = []
combi_list = getCombiList2([], K, R, 0, combi_list)
for n in range(N):
    for k in combi_list:
        mp = getMarginalProbability(n, k, alpha, mu)
        print('mp({0})({1}) = {2}'.format(n, k, mp))

mp(0)([0, 0]) = 0.647434997144275
mp(0)([0, 1]) = 0.19512560508599947
mp(0)([0, 2]) = 0.033706918344990316
mp(0)([1, 0]) = 0.06380906903364138
mp(0)([1, 1]) = 0.04587886108068126
mp(0)([1, 2]) = 0.01404454931041263
mp(1)([0, 0]) = 0.5925769875378033
mp(1)([0, 1]) = 0.17855303689971253
mp(1)([0, 2]) = 0.03089800848290779
mp(1)([1, 0]) = 0.10209451045382623
mp(1)([1, 1]) = 0.07340617772909001
mp(1)([1, 2]) = 0.02247127889666021
mp(2)([0, 0]) = 0.31572146849807575
mp(2)([0, 1]) = 0.17274795651807534
mp(2)([0, 2]) = 0.05266705991404736
mp(2)([1, 0]) = 0.15999550574422072
mp(2)([1, 1]) = 0.19774725429060983
mp(2)([1, 2]) = 0.10112075503497092
mp(3)([0, 0]) = 0.39114069829499176
mp(3)([0, 1]) = 0.31122721271874376
mp(3)([0, 2]) = 0.07820005056037753
mp(3)([1, 0]) = 0.09269402544872338
mp(3)([1, 1]) = 0.09437937136597288
mp(3)([1, 2]) = 0.0323586416111907


In [36]:
#def getMarginalProbabilitySet(alpha, mu): #全ての周辺分布を求める
#    mp_set= np.zeros((N, K1+1, K2+1)) #周辺分布
#    for n in range(N):
#        for k1 in range(K1+1):
#            for k2 in range(K2+1):
#                mp = getMarginalProbability(n, [k1, k2], alpha, mu)
#                mp_set[n,k1,k2] = mp
#    return mp_set

In [37]:
def getMarginalProbabilitySet(alpha, mu): #全ての周辺分布を求める
    NK = [k+1 for k in K]
    NK.insert(0, N)
    mp_set= np.zeros(NK) #周辺分布
    combi_list = []
    combi_list = getCombiList2([], K, R, 0, combi_list)
    for n in range(N):
        for k in combi_list:
            nk = []
            nk.append([n])
            for i in k: #ndarrayでリストで要素を指定するときには[[n],[k1],[k2]]のように[]をつけて、タプルにする
                nk.append([i])
            mp = getMarginalProbability(n, k, alpha, mu)
            mp_set[tuple(nk)] = mp
    return mp_set

In [38]:
mp_set = getMarginalProbabilitySet(alpha, mu) #[(0,0),(0,1),(0,2)],[(1,0),(1,1),(1,2)]
mp_set

array([[[0.647435  , 0.19512561, 0.03370692],
        [0.06380907, 0.04587886, 0.01404455]],

       [[0.59257699, 0.17855304, 0.03089801],
        [0.10209451, 0.07340618, 0.02247128]],

       [[0.31572147, 0.17274796, 0.05266706],
        [0.15999551, 0.19774725, 0.10112076]],

       [[0.3911407 , 0.31122721, 0.07820005],
        [0.09269403, 0.09437937, 0.03235864]]])

In [39]:
#平均系内人数を求める
#def getEXP(mp_set):
#    exp = np.zeros((N, R))
#    for n in range(N):
#        for k1 in range(K1+1): #課題：クラスの増減に対応する
#            for k2 in range(K2+1):
#                exp[n,0] += k1 * mp_set[n,k1,k2]
#                exp[n,1] += k2 * mp_set[n,k1,k2]
#    return exp

In [40]:
#平均系内人数を求める
def getEXP(mp_set):
    exp = np.zeros((N, R))
    combi_list = []
    combi_list = getCombiList2([], K, R, 0, combi_list)
    for n in range(N):
        for k in combi_list:
            nk = []
            nk.append([n])
            for i in k: #ndarrayでリストで要素を指定するときには[[n],[k1],[k2]]のように[]をつけて、タプルにする
                nk.append([i])
            for r in range(R):
                exp[n,r] += k[r] * mp_set[tuple(nk)]
    return exp

In [41]:
exp = getEXP(mp_set)
exp

array([[0.12373248, 0.3365074 ],
       [0.19797197, 0.35869779],
       [0.45886352, 0.67807084],
       [0.21943204, 0.62672397]])

In [42]:
#スループット算出
#def getThroughput(alpha, mu):
#    tp = np.zeros((N, R))
#    for n in range(N):
#        for r in range(R): 
#            if r == 0:#課題：クラスの増減に対応
#                tp[n,r] = alpha[n,r] * getConvolution(N-1, alpha, mu, [K1-1, K2], type_list[N-1]) / GK
#                #getConvolutionはn=Nのときを求める
#            elif r == 1:
#                tp[n,r] = alpha[n,r] * getConvolution(N-1, alpha, mu, [K1, K2-1], type_list[N-1]) / GK
#    return tp

In [43]:
#スループット算出
def getThroughput(alpha, mu):
    tp = np.zeros((N, R))
    for n in range(N):
        for r in range(R):
            r1 = np.zeros(R, dtype = int)
            r1[r] = 1
            tp[n,r] = alpha[n,r] * getConvolution(N-1, alpha, mu, np.array(K) - r1, type_list[N-1]) / GK
    return tp

In [44]:
tp = getThroughput(alpha, mu)
tp

array([[0.09143002, 0.13056749],
       [0.03657201, 0.052227  ],
       [0.03657201, 0.03917025],
       [0.018286  , 0.03917025]])

In [45]:
#利用率算出
tp[:,0] / mu[:,0]

array([0.09143002, 0.14628803, 0.29257605, 0.21943204])

In [46]:
mu[:,0]

array([1.        , 0.25      , 0.125     , 0.08333333])

In [47]:
#利用率算出 P322 式(8.29)lambda / (m * mu) 今回はm = 1 
rho_n = tp / mu
rho_n

array([[0.09143002, 0.26113499],
       [0.14628803, 0.26113499],
       [0.29257605, 0.39170248],
       [0.21943204, 0.62672397]])

In [48]:
#kのリスト(重複組合せ)を作成：ここでは使わない
import itertools #重複組合せを求める
def getCombi(N,K):
    #s = combinations_count(N, K)#重複組み合わせ
    l = [i for i in range(N)]
    p_list = list(itertools.combinations_with_replacement(l, K))
    combi = [[0 for i in range(N)] for j in range(len(p_list))]
    for ind, p in enumerate(p_list):
        for k in range(K):
            combi[ind][p[k]] += 1
    return combi

In [49]:
combi = getCombi(R, K1+K2)
combi
#求めたいのはこれではない。[0,0],[1,0],[1,1],[0,1][1,2]のように総数は一定でないが、制限での総組み合わせ
#制限をリストで渡して、再帰で計算すればいい？

[[3, 0], [2, 1], [1, 2], [0, 3]]

## 推移確率行列の算出

In [50]:
from numpy.linalg import solve 
def getCloseTraffic(p):
    e = np.eye(len(p)-1) #次元を1つ小さくする
    #print(e)
    pe = p[1:len(p), 1:len(p)].T - e #行と列を指定して次元を小さくする
    #print(pe)
    lmd = p[0, 1:len(p)] #0行1列からの値を右辺に用いる
    #print(lmd)
    slv = solve(pe, lmd * (-1))
    alpha = np.insert(slv, 0, 1.0) #α1=1を追加
    return alpha

In [51]:
# QN&MC P333 Example 8.4
p = np.array([[0,0.5,0.5,0],[0,0,0,1],[0,0,0,1],[1,0,0,0]])
print(p)
alpha = getCloseTraffic(p)
alpha #結果OK

[[0.  0.5 0.5 0. ]
 [0.  0.  0.  1. ]
 [0.  0.  0.  1. ]
 [1.  0.  0.  0. ]]


array([1. , 0.5, 0.5, 1. ])

In [52]:
#QN&MC P370 Example 8.10
p = np.array([[0, 0.5, 0.5, 0],[0.5, 0, 0, 0.5],[0.5, 0, 0, 0.5],[0, 0.4, 0.4, 0.2]])
print(p)
alpha = getCloseTraffic(p)
alpha #結果OK

[[0.  0.5 0.5 0. ]
 [0.5 0.  0.  0.5]
 [0.5 0.  0.  0.5]
 [0.  0.4 0.4 0.2]]


array([1.  , 1.  , 1.  , 1.25])

In [53]:
#マルチクラスの場合
p = np.array([[0,0.5,0.5,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,1,0,0,0,0],[1,0,0,0,0,0,0,0],
             [0,0,0,0,0,0.5,0.5,0],[0,0,0,0,0,0,0,1],[0,0,0,0,0,0,0,1],[0,0,0,0,1,0,0,0]])
#alpha = getCloseTraffic(p)
#alpha
#マルチクラスでクラススイッチなしの場合は、それぞれのクラスでα1=1としないといけないので、クラススイッチない場合はクラス一つずつ実施して到着率αを得る方が良い

In [54]:
#推移確率自動生成
p = np.random.rand(N, N)
print(np.sum(p, axis=1))
print(np.sum(p[0]))
for i, val in enumerate(np.sum(p, axis=1)): #正規化 axis=1で行和
    p[i] /= val
print(np.sum(p, axis=1))
p

[1.86810187 0.59399777 2.16848436 2.78429651]
1.8681018651945136
[1. 1. 1. 1.]


array([[0.02462896, 0.40301935, 0.28771739, 0.2846343 ],
       [0.43594239, 0.0789044 , 0.48063551, 0.0045177 ],
       [0.12557339, 0.39642881, 0.10381272, 0.37418508],
       [0.21685482, 0.32373105, 0.18104176, 0.27837238]])

In [55]:
    #同値類を求める関数
    def getEquivalence(th, roop, p):
        list_number = 0 #空のリストを最初から使用する

        #1. 空のリストを作成して、ノードを追加しておく(作成するのはノード数分)
        equivalence = [[] for i in range(len(p))] 
        
        #2. Comunicationか判定して、Commnicateの場合リストに登録
        for ix in range(roop):
            p = np.linalg.matrix_power(p.copy(), ix+1) #累乗
            for i in range(len(p)):
                for j in range(i+1, len(p)):
                    if(p[i][j] > th and p[j][i] > th): #Communicateの場合
                        #3. Communicateの場合登録するリストを選択
                        find = 0 #既存リストにあるか
                        for k in range(len(p)):
                            if i in equivalence[k]: #既存のk番目リストに発見(iで検索)
                                find = 1 #既存リストにあった
                                if j not in equivalence[k]: #jがリストにない場合登録
                                    equivalence[k].append(j)        
                                break
                            if j in equivalence[k]: #既存のk番目リストに発見(jで検索)
                                find = 1 #既存リストにあった
                                if i not in equivalence[k]:
                                    equivalence[k].append(i)        
                                break
                        if(find == 0):#既存リストにない
                            equivalence[list_number].append(i)
                            if(i != j):
                                equivalence[list_number].append(j)
                            list_number += 1

        #4. Communicateにならないノードを登録
        for i in range(len(p)):
            find = 0
            for j in range(len(p)):
                if i in equivalence[j]:
                    find = 1
                    break
            if find == 0:
                equivalence[list_number].append(i)
                list_number += 1

        #5. エルゴード性の確認(class数が1のとき)
        class_number = 0
        for i in range(len(p)):
            if len(equivalence[i]) > 0:
                class_number += 1

        return equivalence, class_number

In [56]:
getEquivalence(0, 2, p)

([[0, 1, 2, 3], [], [], []], 1)

In [57]:
#クラス数分推移確率行列を生成して、それぞれの到着率を返す関数
def getProbArrival(N, R):
    pr = np.zeros((R*N, R*N))
    alpha = np.zeros((R, N))
    for r in range(R):
        class_number = 0
        while class_number != 1: #エルゴード性を持つか確認
            p = np.random.rand(N, N)
            for i, val in enumerate(np.sum(p, axis=1)): #正規化 axis=1で行和
                p[i] /= val
            for i in range(N):#推移確率のマージ
                for j in range(N):
                    pr[r*N+i,r*N+j] = p[i,j]
            equivalence, class_number = getEquivalence(0, 5, p)
            if class_number == 1: #クラス数が1(エルゴード性を持つ)
                break
        alpha_r = getCloseTraffic(p)
        for i, val in enumerate(alpha_r): #到着率を配列alphaに格納
            alpha[r,i] = val
            #print('r = {0}, i = {1}, val = {2}'.format(r,i,val))
    return alpha, pr

In [58]:
#データの保存
def saveCSVi(df, fname):
    pdf = pd.DataFrame(df) #データフレームをpandasに変換
    pdf.to_csv(fname, index=True) #index=Falseでインデックスを出力しない

In [59]:
import pandas as pd
alpha, pr = getProbArrival(4, 2)
print(pr)
saveCSVi(pr, './tpr/tprNR_'+str(N)+'_'+str(R)+'.csv')
alpha
# 推移確率行列をランダムに生成して、エルゴード性を持つチェックをして、到着率を算出可能
# 課題：生成した推移確率を残しておきたい

[[0.30377626 0.32476654 0.23425813 0.13719906 0.         0.
  0.         0.        ]
 [0.15576384 0.20032261 0.27903592 0.36487763 0.         0.
  0.         0.        ]
 [0.19564219 0.33614517 0.15313222 0.31508042 0.         0.
  0.         0.        ]
 [0.3418407  0.06179193 0.00679301 0.58957437 0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.13525677 0.41941092
  0.3691736  0.0761587 ]
 [0.         0.         0.         0.         0.22187127 0.2458088
  0.40627209 0.12604784]
 [0.         0.         0.         0.         0.16280226 0.26891641
  0.30876327 0.25951805]
 [0.         0.         0.         0.         0.26208431 0.30542858
  0.36060725 0.07187986]]


array([[1.        , 0.73715915, 0.53071124, 1.39705861],
       [1.        , 1.55580587, 1.87504976, 0.81764623]])

In [60]:
def getDevideK(K, R):#各クラスの人数を均等に分ける
    dev_k = []
    for i in range(R-1):
        dev_k.append(K // R)
    dev_k.append(K - sum(dev_k))
    return dev_k

In [61]:
getDevideK(10, 4)

[2, 2, 2, 4]

In [62]:
K = 10
R = 4
dev_k = [(K + i) // R for i in range(R)]
dev_k

[2, 2, 3, 3]

## P304 Example 7.7 Open BCMP Network

In [63]:
N = 3
R = 2
type_list = [2, 4, 4]
#lmd = np.array([[1,1],[0,0],[0,0]]) #ノード1にしか到着しないので書いていない
lmd = np.array([[1,0,0],[1,0,0]])#外部からの到着率
p0 = np.array([[1,0,0],[1,0,0]]) #外部からの到着確率
lmd_p = lmd * p0 #トラフィック方程式のλに相当するのがlmd_p
mu = np.array([[8, 24],[12, 32],[16, 36]])
#p = np.array([[0,1,0,0,0,0,0,0],[0.3,0,0.4,0.3,0,0,0,0],[0,0.6,0,0.4,0,0,0,0],[0,0.5,0.5,0,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0.1,0,0.3,0.6],[0,0,0,0,0,0.7,0,0.3],[0,0,0,0,0,0.4,0.6,0]])
p = np.array([[0,0.4,0.3,0,0,0],[0.6,0,0.4,0,0,0],[0.5,0.5,0,0,0,0],[0,0,0,0,0.3,0.6],[0,0,0,0.7,0,0.3],[0,0,0,0.4,0.6,0]])

In [64]:
#トラフィック方程式の計算
#(P^T-E)x = -λ
#ie = np.eye(N*R)#単位行列の生成
#lmd_pv = np.reshape(lmd_p, N*R) #行列からベクトルに変換
#lmd_pv *= -1 #右辺に移項
#from numpy.linalg import solve
#solve(p.T - ie, lmd_pv)

In [65]:
from numpy.linalg import solve
def getTraffic(N, R, p, lmd):
    ie = np.eye(N*R) #単位行列の生成
    lmd_pv = np.reshape(lmd_p, N*R) #行列からベクトルに変換
    lmd_pv *= -1 #右辺に移項
    return solve(p.T - ie, lmd_pv)#(P^T-E)x = -λ

In [66]:
alpha = getTraffic(N, R, p, lmd)
alpha_m = np.reshape(alpha,[R,N])
alpha_m

array([[ 3.33333333,  2.29166667,  1.91666667],
       [10.        ,  8.04878049,  8.41463415]])

In [67]:
#utilization P306
rho = alpha_m / mu.T
print(rho)
rho_column = np.sum(rho, axis=0)
print(rho_column)

[[0.41666667 0.19097222 0.11979167]
 [0.41666667 0.25152439 0.23373984]]
[0.83333333 0.44249661 0.3535315 ]


In [68]:
# Mean number of jobs P306
L = np.zeros((R, N))
for i in range(R):
    for j in range(N):
        L[i,j] = rho[i,j] / (1 - rho_column[j])
L

array([[2.5       , 0.34254899, 0.18530163],
       [2.5       , 0.45116208, 0.36156416]])

In [69]:
# Marginal Probabilities
for i in range(N):
    for j in range(4):
        print('pi({0}, {1}) = {2}'.format(i, j, (1 - rho_column[i])*rho_column[i]**j))

pi(0, 0) = 0.16666666666666718
pi(0, 1) = 0.13888888888888923
pi(0, 2) = 0.11574074074074095
pi(0, 3) = 0.09645061728395074
pi(1, 0) = 0.5575033875338757
pi(1, 1) = 0.24669336042212892
pi(1, 2) = 0.10916097630467668
pi(1, 3) = 0.048303362228314294
pi(2, 0) = 0.6464684959349597
pi(2, 1) = 0.2285469796985507
pi(2, 2) = 0.08079855748235086
pi(2, 3) = 0.028564835553021115


### The Mean Value Analysis

### P333 Example8.3 (Single Class Closed Queue)

In [73]:
#P333 Example8.3
N = 4
K = 6
R = 1
mu = np.array([1/0.02, 1/0.2, 1/0.4, 1/0.6])
m = np.array([1, 1, 1, 1])
alpha = np.array([1.0, 0.4, 0.2, 0.1])

In [99]:
#Step 1
L = np.zeros(N) #平均形内人数
T = np.zeros(N) #平均系内時間
lmd = 0 #全体スループット
lmd_n = np.zeros(N) #各ノードのスループット
G = 1
Gk1 = 1 #G(K-1)を入れておく
rho = np.zeros(N) #利用率

In [100]:
#Step 2
for k in range(K): #系内人数分くりかえす
    #Step 2.1 P330 (8.38) 系内時間に更新
    for n in range(N):
        T[n] = (1 + L[n]) / mu[n]
    
    #Step 2.2 P330 (8.39) 全体スループットを求める
    sum_at = 0
    for n in range(N):
        sum_at += alpha[n] * T[n]
        lmd = (k + 1) / sum_at
        
    #P330 (8.40) 各ノードのスループット
    for n in range(N):
        lmd_n[n] = alpha[n] * lmd
    
    #P330 (8.42) 正規化定数の更新
    if k == K-1: #G(K-1)を確保
        GK1 = G
    G /= lmd
    
    #Step 2.3 系内人数の更新
    for n in range(N):
        L[n] = lmd * T[n] * alpha[n]
        
# 利用率の計算 P317 (8.15)
for n in range(N):
    rho[n] = alpha[n] / (m[n] * mu[n]) * GK1 / G

#P332 Table 8.4    
print('Mean Respons Time T : {0}'.format(T))
print('Throughput Lambda : {0}'.format(lmd_n))
print('Mean number of job L : {0}'.format(L))
print('Utilization : rho {0}'.format(rho))

print('Throughput Lambda : {0}'.format(lmd))
print('Normalization Constant G : {0}'.format(G))

Mean Respons Time T : [0.02455839 0.56982739 1.13965479 1.24428379]
Throughput Lambda : [9.91983634 3.96793453 1.98396727 0.99198363]
Mean number of job L : [0.24361526 2.26103779 2.26103779 1.23430916]
Utilization : rho [0.19839673 0.79358691 0.79358691 0.59519018]
Throughput Lambda : 9.919836337154358
Normalization Constant G : 5.756224000000001e-06


In [101]:
#定常分布の算出 P332
k_n = [3,1,1,1]
sum = 1
for n in range(N):
    sum *= (alpha[n] / mu[n])**k_n[n]
pi = sum / G
print('pi({0}) = {1}'.format(k_n, pi))

pi([3, 1, 1, 1]) = 0.0005336831923149621
